In [1]:
import os
import time
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
from torch.autograd import Variable

### System properties and libs currently in use
- We have developed using python 3.5.x, pytorch 0.2.1
- No significant attention was given to backwards compatibility

In [2]:
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
print('Active CUDA Device: GPU', torch.cuda.current_device())

__Python VERSION: 3.5.3 |Anaconda 4.4.0 (64-bit)| (default, Mar  6 2017, 11:58:13) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
__pyTorch VERSION: 0.2.0_4
__CUDA VERSION
__CUDNN VERSION: 6021
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0


### Utilities
- Saving images and models

In [3]:
def save_images(netG, fixed_noise, outputDir,epoch):
    '''
    Generates a batch of images from the given 'noise'.
    Saves 64 of the generated samples to 'outputDir' system path.
    Inputs are the network (netG), a 'noise' input, system path to which images will be saved (outputDir) and current 'epoch'.
    '''
    noise = Variable(fixed_noise)
    netG.eval()
    fake = netG(noise)
    netG.train()
    vutils.save_image(fake.data[0:64,:,:,:],'%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)

def save_models(netG, netD, outputDir, epoch):
    '''
    Saves model state dictionary for generator and discriminator networks.
    Inputs are the networks (netG, netD), the system path in which to save(outputDir) and the current 'epoch'.
    '''
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [4]:
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


### Output Directory
This is where images will be saved to.

If directory does not exist, it is created.

In [5]:
outputDir = 'outputdir_train_classifier_lotufo'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))

OS error: [Errno 17] File exists: 'outputdir_train_classifier_lotufo'


### Dataset definition and hyperparameter setting
- Changing dataset name alters network architecture parameters
- Currently supporting few datasets
- Hyperparameters defined according to Radford et al. (2015)

In [6]:
batch_size = 128

chosen_dataset = 'MNIST'

datasets = {
    'MNIST': torchvision.datasets.MNIST,
    'CIFAR10': torchvision.datasets.CIFAR10,
    'ANIME': '/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
}

dataset = datasets[chosen_dataset]

In [7]:
possible_parameters = {
    'MNIST': {
        'ndf': 64,
        'ngf': 64,
        'nz': 100,
        'nc': 1,
        'imageSize': 64,
        'n_classes' : 10,
        'ngpu': 1,
    },
    'CIFAR10': {
        'ndf': 64,
        'ngf': 64,
        'nz': 100,
        'nc': 3,
        'imageSize' : 64,
        'n_classes' : 10,
        'ngpu' : 1,
    },
    'ANIME': {
        'nc' : 3,
        'ngpu' : 1,
        'nz' : 100,
        'ngf' : 64,
        'ndf' : 64,
        'imageSize' : 64,
        'n_classes' : 1
    }
}

In [8]:
ngf = possible_parameters[chosen_dataset]['ngf']
ndf = possible_parameters[chosen_dataset]['ndf']
nz = possible_parameters[chosen_dataset]['nz']
nc = possible_parameters[chosen_dataset]['nc']
imageSize = possible_parameters[chosen_dataset]['imageSize']
n_classes = possible_parameters[chosen_dataset]['n_classes']
ngpu = possible_parameters[chosen_dataset]['ngpu']

## Creating the Dataset!

In [9]:
if dataset == 'ANIME':
    dataset = torchvision.datasets.ImageFolder(
        root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
        transform=transforms.Compose([
                transforms.Scale((imageSize, imageSize)),
                transforms.ToTensor(),
            ])
    )
else:
    transform = transforms.Compose([
                    transforms.Scale((imageSize, imageSize)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
                ]) 
    dataset_done = dataset('./datasets', train=True, download=True, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset_done, batch_size=batch_size, shuffle=True, num_workers=4)
print('Dataloader length:', len(dataloader))
print("Dataset:", dataloader.dataset)


Dataloader length: 469
Dataset: <torchvision.datasets.mnist.MNIST object at 0x7f6b16d912e8>


## Definição dos modelos
- Model is a DCGAN
- Images are sized (nc, 64, 64)

In [10]:
class _netD_DCGAN(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf, n_classes):
        super(_netD_DCGAN, self).__init__()
        self.ngpu = ngpu
        self.conv1 = nn.Conv2d(in_channels = nc, out_channels = ndf, kernel_size=4, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels = ndf, out_channels = ndf*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(in_channels = ndf*2, out_channels = ndf*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(in_channels = ndf*4, out_channels = ndf*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ndf * 8)
        
        self.final_conv = nn.Conv2d(in_channels=ndf*8, out_channels=n_classes+1,kernel_size=4,stride=1,padding=0,bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, inplace=True)
        x = F.leaky_relu(self.batch2(self.conv2(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch3(self.conv3(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch4(self.conv4(x)), 0.2, inplace=True)
        
        x = self.final_conv(x)
        return(x)

In [11]:
class _netG_DCGAN(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf):
        super(_netG_DCGAN, self).__init__()
        self.ngpu = ngpu
        self.convt1 = nn.ConvTranspose2d(in_channels=nz, out_channels=ngf * 8, kernel_size=4, stride=1, padding=0, bias=False)
        self.batch1 = nn.BatchNorm2d(ngf*8)
        self.convt2 = nn.ConvTranspose2d(in_channels=ngf * 8, out_channels=ngf * 4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ngf*4)
        self.convt3 = nn.ConvTranspose2d(in_channels=ngf * 4, out_channels=ngf * 2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ngf*2)
        self.convt4 = nn.ConvTranspose2d(in_channels=ngf*2, out_channels=ngf, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ngf)
        
        self.final_convt = nn.ConvTranspose2d(in_channels=ngf, out_channels=nc, kernel_size=4, stride=2, padding=1, bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.batch1(self.convt1(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch2(self.convt2(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch3(self.convt3(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch4(self.convt4(x)), 0.2, inplace=True)
        
        x = self.final_convt(x)
        x = F.tanh(x)
        return (x)

In [12]:
netG = _netG_DCGAN(ngpu, nz, nc, ngf)
netD = _netD_DCGAN(ngpu, nz, nc, ndf, n_classes)

## Inicializador de pesos

In [13]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [14]:
netG.apply(weights_init)
netD.apply(weights_init)
print(netG, '\n', netD)

_netG_DCGAN (
  (convt1): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (batch1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (convt2): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (convt3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (convt4): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (final_convt): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
) 
 _netD_DCGAN (
  (conv1): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch2): BatchNorm2d(128, eps=

## Losses
- Binary Cross-Entropy is used to differentiate real and fake images
- Class loss should be Cross-Entropy

In [15]:
criterion = nn.CrossEntropyLoss()

## Sizes of the tensors

In [16]:
input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print('Input images size:', input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
print('Code size:', noise.size())

Input images size: torch.Size([128, 3, 64, 64])
Code size: torch.Size([128, 100, 1, 1])


In [17]:
label = torch.LongTensor(batch_size,n_classes)
print('Label size:', label.size())
fake_label = 10
real_label = 1

Label size: torch.Size([128, 10])


## Broadcast to GPU

In [18]:
if use_gpu:
    netD.cuda()
    netG.cuda()
    criterion = criterion.cuda()
    input,label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

## Optimizer Parameters
- Following the lead of Radford et al., 2015:

    <b>
    1. beta1 = 0.5
    2. beta2 = 0.999
    3. lr = 0.0002
    </b>

In [19]:
beta1, beta2 = 0.5, 0.999
lr = 2.0e-4

optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))


## Treinamento 

In [20]:
def train_gan(num_epochs, dataloader, netD, netG, outputDir,
              real_labelSmooth=0, epoch_interval=100, D_steps=1, G_steps=1, decay = 0.05):
    
    # This validation is subjective. WGAN-GP uses 100 steps on the critic (netD).
    assert D_steps < 5, "Keep it low, D_steps is too high."
    assert G_steps < 3, "Keep it low, G_steps is too high."
    #assert batch_size % D_steps == 0, "Use batch_size multiple of D_steps."
    real_label = 1
    print('Lets train!')
    for epoch in range(num_epochs):
        start_iter = time.time()  
        D_x = 0
        D_G_z1 = 0
        D_G_z2 = 0
        errD_acum = 0
        errG_acum = 0
        
        print('In epoch = ', epoch, 'real_label_smooth = ',real_labelSmooth )
        for batch, data in enumerate(dataloader, 0):
            if (epoch == 0 and batch == 0):
                    vutils.save_image(data[0][0:64,:,:,:], '%s/real_samples.png' % outputDir, nrow=8)
            if epoch < 25:   
                real_labelSmooth = np.maximum(real_labelSmooth * (1 - 0.05*epoch), 0)
            else:
                real_labelSmooth = 0
            for step in range(D_steps):
                #############################################################
                # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
                # 1A - Train the detective network in the Real Dataset
                #############################################################
                netD.zero_grad()
                start = step*(int(data[0].size()[0]/D_steps))
                end = (step+1)*int(data[0].size()[0]/D_steps)
                
                real_cpu = data[0][start:end]
                real_cpu = real_cpu.cuda()
                batch_size = real_cpu.size(0)
                if np.random.random_sample() > real_labelSmooth:
                    target = data[1][start:end].long().cuda()
                else:
                     target = torch.from_numpy(np.random.randint(0, n_classes, batch_size)).type(torch.LongTensor).cuda()
                
                input, label = Variable(real_cpu), Variable(target)

                output = netD(input)
                errD_real = criterion(output.squeeze(),label)
                errD_real.backward()
                
                D_x += output.data.mean()
                
                #######################################################
                # 1B - Train the detective network in the False Dataset
                #######################################################
                
                noise = Variable(torch.FloatTensor(batch_size, nz, 1, 1).normal_(0,1).cuda())
                fake = netG(noise)
                label = Variable(torch.ones(batch_size).long().fill_(fake_label).cuda())
                output = netD(fake.detach()) # ".detach()" to avoid backprop through G
                errD_fake = criterion(output.squeeze(), label)
                errD_fake.backward() # gradients for fake and real data will be accumulated
                
                D_G_z1 += output.data.mean()
                errD_acum += errD_real.data[0] + errD_fake.data[0]
                optimizerD.step()

            for step in range(G_steps):
                ####################################################################################
                # (2) Update G network: maximize log(D(G(z)))
                # Train the faker with the output from the Detective (but don't train the Detective)
                ####################################################################################
                
                netG.zero_grad()
                label = Variable(torch.from_numpy(np.random.randint(0, n_classes, batch_size)).type(torch.LongTensor).cuda())
                output = netD(fake)
                errG = criterion(output.squeeze(), label)
                errG.backward()
                
                D_G_z2 += output.data.mean()
                errG_acum += errG.data[0]
                optimizerG.step()

        print('epoch = ',epoch)

        end_iter = time.time()        

        print('[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
              % (epoch, num_epochs, errD_acum/D_steps, errG_acum/G_steps, D_x, D_G_z1, D_G_z2, end_iter-start_iter))

        #Save a grid with the pictures from the dataset, up until 64
        save_images(netG = netG, fixed_noise=  fixed_noise, outputDir = outputDir, epoch = epoch)

        if epoch % epoch_interval == 0:
            # do checkpointing
            save_models(netG = netG, netD = netD, outputDir = outputDir, epoch = epoch)
            

outputDir = 'outputdir_train_classifier_0d25'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))

num_epochs = 25
real_labelSmooth = 0.25
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
    
num_epochs = 100
real_labelSmooth = 0.15

train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth)

outputDir = 'outputdir_train_classifier_0d20'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
    
num_epochs = 100
real_labelSmooth = 0.20

train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth)

outputDir = 'outputdir_train_classifier_0d10'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
    
num_epochs = 100
real_labelSmooth = 0.10

train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d1'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.1
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.15
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d20'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.20
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d25'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.25
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d30'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.30
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.15
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d10'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.10
decay = 0.025
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d018_label_0d10'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.10
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

outputDir = 'outputdir_train_classifier_decay_0d025_label_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.10
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

In [21]:
outputDir = 'outputdir_train_classifier_decay_0d018_label_0d15'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.15
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.15
epoch =  0
[0/40] Loss_D: 683.4500 Loss_G: 3620.1620 D(x): -400.8651 D(G(z)): -445.6175 / -453.9886 Elapsed 46.25 s
In epoch =  1 real_label_smooth =  0.15
epoch =  1
[1/40] Loss_D: 174.6914 Loss_G: 4018.7320 D(x): -521.3670 D(G(z)): -552.2424 / -554.4528 Elapsed 44.45 s
In epoch =  2 real_label_smooth =  5.35133500423e-12
epoch =  2
[2/40] Loss_D: 147.4305 Loss_G: 4895.4638 D(x): -809.0297 D(G(z)): -841.2120 / -843.0898 Elapsed 44.47 s
In epoch =  3 real_label_smooth =  1.85438141721e-33
epoch =  3
[3/40] Loss_D: 115.6441 Loss_G: 5433.3562 D(x): -960.9524 D(G(z)): -997.7818 / -1000.2700 Elapsed 44.49 s
In epoch =  4 real_label_smooth =  1.46445228692e-66
epoch =  4
[4/40] Loss_D: 152.1810 Loss_G: 5542.4809 D(x): -1138.0070 D(G(z)): -1168.1835 / -1171.3924 Elapsed 44.49 s
In epoch =  5 real_label_smooth =  5.18655664962e-112
epoch =  5
[5/40] Loss_D: 116.6636 Loss_G: 5959.3384 D(x): -1339.5754 D(G(z)): -1382.4511 / -1385.3886 Elapsed 

In [22]:
outputDir = 'outputdir_train_classifier_decay_0d018_label_0d20'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.20
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.2
epoch =  0
[0/40] Loss_D: 1008.1055 Loss_G: 2696.7562 D(x): -218.5605 D(G(z)): -273.4744 / -277.5046 Elapsed 45.66 s
In epoch =  1 real_label_smooth =  0.2
epoch =  1
[1/40] Loss_D: 99.9920 Loss_G: 5067.1949 D(x): -851.7914 D(G(z)): -1025.8148 / -1038.1201 Elapsed 45.64 s
In epoch =  2 real_label_smooth =  7.13511333898e-12
epoch =  2
[2/40] Loss_D: 63.8840 Loss_G: 6540.9229 D(x): -1260.8770 D(G(z)): -1458.2403 / -1464.9842 Elapsed 45.60 s
In epoch =  3 real_label_smooth =  2.47250855628e-33
epoch =  3
[3/40] Loss_D: 40.4819 Loss_G: 7146.6697 D(x): -1499.2905 D(G(z)): -1719.7395 / -1725.8787 Elapsed 45.69 s
In epoch =  4 real_label_smooth =  1.95260304923e-66
epoch =  4
[4/40] Loss_D: 46.8678 Loss_G: 7307.9216 D(x): -1478.4319 D(G(z)): -1714.4891 / -1724.3263 Elapsed 45.71 s
In epoch =  5 real_label_smooth =  6.91540886616e-112
epoch =  5
[5/40] Loss_D: 4.4276 Loss_G: 8256.0018 D(x): -1581.1736 D(G(z)): -2104.8753 / -2110.7320 Elapsed 

In [23]:
outputDir = 'outputdir_train_classifier_decay_0d018_label_0d25'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.25
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.25
epoch =  0
[0/40] Loss_D: 836.9469 Loss_G: 4431.8725 D(x): 2238.3878 D(G(z)): 1584.0503 / 1528.8859 Elapsed 45.71 s
In epoch =  1 real_label_smooth =  0.25
epoch =  1
[1/40] Loss_D: 86.3262 Loss_G: 4581.8311 D(x): 1653.3874 D(G(z)): 1242.0588 / 1226.4449 Elapsed 45.66 s
In epoch =  2 real_label_smooth =  8.91889167372e-12
epoch =  2
[2/40] Loss_D: 12.4761 Loss_G: 6448.4066 D(x): 55.5296 D(G(z)): -19.6827 / -25.1500 Elapsed 45.61 s
In epoch =  3 real_label_smooth =  3.09063569535e-33
epoch =  3
[3/40] Loss_D: 4.4477 Loss_G: 7580.6765 D(x): -373.3870 D(G(z)): -709.0254 / -715.4986 Elapsed 45.76 s
In epoch =  4 real_label_smooth =  2.44075381154e-66
epoch =  4
[4/40] Loss_D: 29.8472 Loss_G: 7073.6220 D(x): 32.5563 D(G(z)): -148.9114 / -153.4238 Elapsed 45.66 s
In epoch =  5 real_label_smooth =  8.6442610827e-112
epoch =  5
[5/40] Loss_D: 4.2858 Loss_G: 8018.0917 D(x): -301.7691 D(G(z)): -576.1989 / -583.0968 Elapsed 45.65 s
In epoch =  6

In [24]:
outputDir = 'outputdir_train_classifier_decay_0d018_label_0d30'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))
num_epochs = 40
real_labelSmooth = 0.30
decay = 0.018
train_gan(num_epochs, dataloader, netD,netG, outputDir, real_labelSmooth, decay = decay)

Lets train!
In epoch =  0 real_label_smooth =  0.3
epoch =  0
[0/40] Loss_D: 1181.7568 Loss_G: 2779.1538 D(x): 3188.0879 D(G(z)): 3023.7077 / 3016.8053 Elapsed 45.64 s
In epoch =  1 real_label_smooth =  0.3
epoch =  1
[1/40] Loss_D: 69.2264 Loss_G: 4987.9738 D(x): 2419.3645 D(G(z)): 1979.3055 / 1964.2221 Elapsed 45.56 s
In epoch =  2 real_label_smooth =  1.07026700085e-11
epoch =  2
[2/40] Loss_D: 3.6412 Loss_G: 6421.1169 D(x): 1858.3558 D(G(z)): 1175.3466 / 1169.9474 Elapsed 45.47 s
In epoch =  3 real_label_smooth =  3.70876283442e-33
epoch =  3
[3/40] Loss_D: 1.9290 Loss_G: 7132.7592 D(x): 1514.8070 D(G(z)): 799.6234 / 795.0361 Elapsed 45.64 s
In epoch =  4 real_label_smooth =  2.92890457385e-66
epoch =  4
[4/40] Loss_D: 1.5477 Loss_G: 7475.5311 D(x): 1284.1282 D(G(z)): 602.5216 / 597.5237 Elapsed 45.59 s
In epoch =  5 real_label_smooth =  1.03731132992e-111
epoch =  5
[5/40] Loss_D: 1.2855 Loss_G: 7757.9888 D(x): 1131.1874 D(G(z)): 410.3077 / 404.3236 Elapsed 45.53 s
In epoch =  6 r